# Ordering


## Warning: This notebook will place live orders

Use a paper trading account (during market hours).


In [1]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=13)

<IB connected to 127.0.0.1:7497 clientId=13>

Create a contract and a market order:

In [2]:
contract = Stock('AAPL', 'SMART', 'USD')
ib.qualifyContracts(contract)

order = MarketOrder('BUY', 100)

placeOrder will place the order order and return a ``Trade`` object right away (non-blocking):

In [3]:
trade = ib.placeOrder(contract, order)

trade

Trade(contract=Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), order=MarketOrder(orderId=11, clientId=13, action='BUY', totalQuantity=100), orderStatus=OrderStatus(status='PendingSubmit'), fills=[], log=[TradeLogEntry(time=datetime.datetime(2019, 3, 19, 18, 22, 46, 736520, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='')])

``trade`` contains the order and everything related to it, such as order status, fills and a log.
It will be live updated with every status change or fill of the order.

In [4]:
ib.sleep(1)
trade.log

[TradeLogEntry(time=datetime.datetime(2019, 3, 19, 18, 22, 46, 736520, tzinfo=datetime.timezone.utc), status='PendingSubmit', message=''),
 TradeLogEntry(time=datetime.datetime(2019, 3, 19, 18, 22, 46, 846973, tzinfo=datetime.timezone.utc), status='PreSubmitted', message=''),
 TradeLogEntry(time=datetime.datetime(2019, 3, 19, 18, 22, 46, 887512, tzinfo=datetime.timezone.utc), status='PreSubmitted', message='Fill 100.0@187.0'),
 TradeLogEntry(time=datetime.datetime(2019, 3, 19, 18, 22, 46, 887512, tzinfo=datetime.timezone.utc), status='Filled', message='')]

``trade`` will also available from ``ib.trades()``:

In [5]:
assert trade in ib.trades()

Likewise for ``order``:

In [6]:
assert order in ib.orders()

Now let's create a limit order with an unrealistic limit:

In [7]:
limitOrder = LimitOrder('BUY', 100, 0.05)
limitTrade = ib.placeOrder(contract, limitOrder)

limitTrade

Trade(contract=Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), order=LimitOrder(orderId=12, clientId=13, action='BUY', totalQuantity=100, lmtPrice=0.05), orderStatus=OrderStatus(status='PendingSubmit'), fills=[], log=[TradeLogEntry(time=datetime.datetime(2019, 3, 19, 18, 22, 47, 788437, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='')])

``status`` will change from "PendingSubmit" to "Submitted":

In [8]:
ib.sleep(1)
assert limitTrade.orderStatus.status == 'Submitted'

In [9]:
assert limitTrade in ib.openTrades()

Let's modify the limit price and resubmit:

In [10]:
limitOrder.lmtPrice = 0.10

ib.placeOrder(contract, limitOrder)

Trade(contract=Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), order=LimitOrder(softDollarTier=SoftDollarTier(), orderId=12, clientId=13, permId=1413558882, action='BUY', totalQuantity=100.0, lmtPrice=0.1, auxPrice=0.0, tif='DAY', ocaType=3, trailStopPrice=1.05, volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='DU772802', clearingIntent='IB', adjustedOrderType='None', cashQty=0.0, dontUseAutoPriceForHedge=True), orderStatus=OrderStatus(status='Submitted', remaining=100.0, permId=1413558882, clientId=13), fills=[], log=[TradeLogEntry(time=datetime.datetime(2019, 3, 19, 18, 22, 47, 788437, tzinfo=datetime.timezone.utc), status='PendingSubmit', message=''), TradeLogEntry(time=datetime.datetime(2019, 3, 19, 18, 22, 47, 895179, tzinfo=datetime.timezone.utc), status='Submitted', message=''), TradeLogEntry(time=datetime.datetime(2019, 3, 19, 18, 22, 48, 826540, tzinfo=datetime.

And now cancel it:

In [11]:
ib.cancelOrder(limitOrder)

Trade(contract=Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), order=LimitOrder(softDollarTier=SoftDollarTier(), orderId=12, clientId=13, permId=1413558882, action='BUY', totalQuantity=100.0, lmtPrice=0.1, auxPrice=0.0, tif='DAY', ocaType=3, trailStopPrice=1.05, volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='DU772802', clearingIntent='IB', adjustedOrderType='None', cashQty=0.0, dontUseAutoPriceForHedge=True), orderStatus=OrderStatus(status='PendingCancel', remaining=100.0, permId=1413558882, clientId=13), fills=[], log=[TradeLogEntry(time=datetime.datetime(2019, 3, 19, 18, 22, 47, 788437, tzinfo=datetime.timezone.utc), status='PendingSubmit', message=''), TradeLogEntry(time=datetime.datetime(2019, 3, 19, 18, 22, 47, 895179, tzinfo=datetime.timezone.utc), status='Submitted', message=''), TradeLogEntry(time=datetime.datetime(2019, 3, 19, 18, 22, 48, 826540, tzinfo=datet

In [12]:
limitTrade.log

[TradeLogEntry(time=datetime.datetime(2019, 3, 19, 18, 22, 47, 788437, tzinfo=datetime.timezone.utc), status='PendingSubmit', message=''),
 TradeLogEntry(time=datetime.datetime(2019, 3, 19, 18, 22, 47, 895179, tzinfo=datetime.timezone.utc), status='Submitted', message=''),
 TradeLogEntry(time=datetime.datetime(2019, 3, 19, 18, 22, 48, 826540, tzinfo=datetime.timezone.utc), status='Submitted', message='Modify'),
 TradeLogEntry(time=datetime.datetime(2019, 3, 19, 18, 22, 48, 841987, tzinfo=datetime.timezone.utc), status='PendingCancel', message='')]

placeOrder is not blocking and will not wait on what happens with the order.
To make the order placement blocking, that is to wait until the order is either
filled or canceled, consider the following:

In [13]:
%%time
order = MarketOrder('BUY', 100)

trade = ib.placeOrder(contract, order)
while not trade.isDone():
    ib.waitOnUpdate()

CPU times: user 6.28 ms, sys: 783 µs, total: 7.06 ms
Wall time: 147 ms


What are our positions?

In [14]:
ib.positions()

[Position(account='DU772802', contract=Stock(conId=9579970, symbol='IWM', exchange='ARCA', currency='USD', localSymbol='IWM', tradingClass='IWM'), position=600.0, avgCost=149.368),
 Position(account='DU772802', contract=Forex('EURUSD', conId=12087792, localSymbol='EUR.USD', tradingClass='EUR.USD'), position=40200.0, avgCost=1.13917845),
 Position(account='DU772802', contract=Stock(conId=265598, symbol='AAPL', exchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), position=1651.0, avgCost=182.08892081698966)]

What's the total of commissions paid today?

In [15]:
sum(fill.commissionReport.commission for fill in ib.fills())

3.0

whatIfOrder can be used to see the commission and the margin impact of an order without actually sending the order:

In [16]:
order = MarketOrder('SELL', 200)
ib.whatIfOrder(contract, order)

OrderState(status='PreSubmitted', initMarginBefore='88525.44', maintMarginBefore='88525.44', equityWithLoanBefore='874455.88', initMarginChange='-8235.86', maintMarginChange='-8235.86', equityWithLoanChange='-1.3299999999580905', initMarginAfter='80289.58', maintMarginAfter='80289.58', equityWithLoanAfter='874454.55', minCommission=1.310026, maxCommission=1.510026, commissionCurrency='USD')

In [17]:
ib.disconnect()